# CASO DE ESTUDIO 1
*Case of study 1*

## Data fetching

### Gastos

In [220]:
import pandas as pd

gastos = pd.read_csv('CasoMarketing-Gasto.csv')
gastos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Semana          48 non-null     int64 
 1   GastoAdwords    48 non-null     object
 2   GastoFacebook   48 non-null     object
 3   GastoInstagram  48 non-null     object
 4   GastoOrganico   48 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.0+ KB


In [221]:
gastos.head()

,Semana,GastoAdwords,GastoFacebook,GastoInstagram,GastoOrganico
0,1,"$58,587.80","$13,166.12","$25,644.37",$0.00
1,2,"$67,295.35","$15,130.57","$25,603.19",$0.00
2,3,"$43,330.19","$14,983.00","$28,962.36",$0.00
3,4,"$60,818.66","$14,294.78","$29,601.45",$0.00
4,5,"$43,593.73","$18,685.01","$29,219.29",$0.00


| **Column name** | **Feature description** |
|----------------:| :--------------------- |
| *Semana*        | Week of the data |
| *GastoAdwords*  | Google advertisement cost |
| *GastoFacebook* | Facebook avertisement cost |
| *GastoInstagram*| Intsgram advertisement cost |
| *GastoOrganico* | Company's page cost |

The table must be cleaned and processed.

In [222]:
gastos.drop('GastoOrganico', axis=1, inplace=True) # description specifies it is always 0

to_float = lambda x: float(x.replace(',', '').replace('$', '')) if isinstance(x, str) else x
gastos = gastos.applymap(to_float)
gastos.set_index('Semana')
gastos.head()

,Semana,GastoAdwords,GastoFacebook,GastoInstagram
0,1,58587.80,13166.12,25644.37
1,2,67295.35,15130.57,25603.19
2,3,43330.19,14983.00,28962.36
3,4,60818.66,14294.78,29601.45
4,5,43593.73,18685.01,29219.29


## Operaciones

All the prospects obtained from marketing

In [223]:
ops = pd.read_csv('CasoMarketing-Operaciones.csv')
ops.head(5)

,Prospecto,Semana,Verificado,Oe,PrioridadProspecto,Canal
0,1,1,Si,6.0,Prioridad Alta,Organico
1,2,1,No,NaN,Prioridad Alta,Organico
2,3,1,Si,5.0,Prioridad Alta,Facebook
3,4,1,Si,14.0,Prioridad Alta,AdWords
4,5,1,Si,14.0,Prioridad Alta,AdWords


| **Column name** | **Feature description** |
|----------------:| :--------------------- |
| *Prospecto*     | Prospect ID |
| *Semana*        | Week of the data |
| *Verificado*    | Wether the prospect provided his/her card information. It is the main KPI with which the marketing meassures new subscriptions |
| *Oe*            | Time (in days) between account opening and prospect verification |
| *PriodidadProspecto* | Priority level of the prospect based on a ML algorithm predicting the probability of the prospect becoming verified|
| *Canal*         | Where did the costumer came from |

The table must be cleaned and processed.

In [224]:
for column in list(ops.columns)[2:]:
    print(ops[column].unique())

['Si' 'No']
[ 6. nan  5. 14. 15. 20.  4.  1.  9.  2. 17.  3.  7. 11.  8. 16. 18. 10.
 21. 13. 12. 19.]
['Prioridad Alta' 'Prioridad Baja' 'Prioridad Media']
['Organico' 'Facebook' 'AdWords' 'Instagram']


In [225]:
ops['Oe'].fillna(0, inplace=True) # From the unique list od values, assuming the NA are actually '0 days'

In [226]:
ops.shape[0]==len(ops['Prospecto'].unique()) # Check for repeated users (True if all unique)

True

In [227]:
ops.drop('Prospecto', axis=1, inplace=True) # There is no value for individual identifiers
ops['Oe'] = ops['Oe'].apply(lambda x: int(x))
ops.set_index('Semana')
ops.head()

,Semana,Verificado,Oe,PrioridadProspecto,Canal
0,1,Si,6,Prioridad Alta,Organico
1,1,No,0,Prioridad Alta,Organico
2,1,Si,5,Prioridad Alta,Facebook
3,1,Si,14,Prioridad Alta,AdWords
4,1,Si,14,Prioridad Alta,AdWords


## Join the tables

In [228]:
data = ops.merge(gastos, on='Semana', how='inner')
data.head()

,Semana,Verificado,Oe,PrioridadProspecto,Canal,GastoAdwords,GastoFacebook,GastoInstagram
0,1,Si,6,Prioridad Alta,Organico,58587.8,13166.12,25644.37
1,1,No,0,Prioridad Alta,Organico,58587.8,13166.12,25644.37
2,1,Si,5,Prioridad Alta,Facebook,58587.8,13166.12,25644.37
3,1,Si,14,Prioridad Alta,AdWords,58587.8,13166.12,25644.37
4,1,Si,14,Prioridad Alta,AdWords,58587.8,13166.12,25644.37
